In [ ]:
import numpy as np

from numpy import random

from os.path import dirname, join

from bokeh.io import curdoc
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource, Slider, TextInput, Toggle, Button, RadioButtonGroup
from bokeh.models import CustomJS, HoverTool
from bokeh.models import PreText, CheckboxGroup, Paragraph
from bokeh.plotting import figure
from bokeh.models.widgets import Tabs, Panel

from bokeh.driving import count

from bokeh.models import Range1d

import time
import serial

useSerial = True

if useSerial == True:
    # Estalish a connection with the CD48 board
    s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000, timeout = 1) # open serial port; port name will 
                                                            # have to be changed on different computers

        
# Initialize the counter setting to those desired for G2 Alignment
s.write('S01000\n'.encode())
s.write('S10100\n'.encode())
s.write('S20010\n'.encode())
s.write('S31100\n'.encode())
s.write('S41010\n'.encode())
s.write('S51110\n'.encode())


# Define a class of objects called Counter. Each counter (0-7) will get a widget checkbox 
# for Port A through Port D   
class Counter(object):
    def __init__(self, portA, portB, portC, portD):
        self.portA = portA
        self.portB = portB
        self.portC = portC
        self.portD = portD
    

# Set a variable for common height of widgets
widgetHeight = 20   


# Create Label Widgets for each Counter Row. Ex) the 0th row will say "Counter 0:"
counter_labels = [None] * 8

for i in range(8):
    counter_labels[i] = PreText(text='Counter: ' + str(i), width = 80, height = widgetHeight)
    
# define width of port labels
portLabelWidth = 30

# Create Label widgets for each Port Column
portA = PreText(text='A: ', width = portLabelWidth, height = widgetHeight)
portB = PreText(text='B: ', width = portLabelWidth, height = widgetHeight)
portC = PreText(text='C: ', width = portLabelWidth, height = widgetHeight)
portD = PreText(text='D: ', width = portLabelWidth, height = widgetHeight)


# create some blank widgets just to "take up space" so the layout looks nicer
blank = PreText(text='', width = 80, height = widgetHeight)
blank2 = PreText(text='', width = 10, height = widgetHeight)
blank3 = PreText(text='', width = 20, height = widgetHeight)

# Create Set Button to send counter information to CD48 board
set_button = Button(
    label='Set Counters',
    disabled=False,
    button_type='success' # 'success', 'info', 'warning', 'danger' or ''
)



# define width of checkbox widgets
CheckboxWidth = 30

# Create the list counters. Each element of counters is an object of the class Counter.
# This means each element of counters will have 4 widget checkboxes, each referred to as portA - portD
# Later, you can call the "activeness"of say counter 4, port D checkbox in this way: 
# counters[4].portD.active   
# If counters[4].portD.active = [0], then this box is checked. 
# If counters[4].portD.active = [], then this box is not checked.

counters = [None] * 8

# loop through 8 times to create 4 widget checkboxes (portA, portB, portC, portD) 
# for each of counters 0 through 7
for i in range(8):
    counters[i] = Counter(CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight))
    

# create a list to store board settings in 
boardSettings = [None] * 13

# the first element of boardSettings will just be the title "Board Settings:"
boardSettings[0] = PreText(text="""Board Settings:""", width = 30, height = widgetHeight)

# for the remaining 12 items, just append blank ' ' which can be changed later
for i in np.arange(12):
    boardSettings[i+1] = PreText(text=""" """, width = 30, height = widgetHeight)


# Define a function that can be called later. This function sends the 'P' command to the board, 
# then determines whether a given counter is currently "watching" a given port. 
# Then set the checkbox values accordingly (blah.active = [0] --> True, blah.active = [] --> False), 
# checkboxes are booleans
def set_checkboxes():
    
    # get the board settings
    s.write('P'.encode())
    time.sleep(0.5)
    settings = s.readlines()
    
    # add the board settings to the list boardSettings
    for i in np.arange(len(settings)):
        boardSettings[i+1].text = str(settings[i])
    
    # loop through 8 times to glean port settings for each counter
    for i in range(8):
        # for debugging purposes
        #print('counter = ', i)
        #print(settings[i][2:3])
        #print(settings[i][3:4])
        #print(settings[i][4:5])
        #print(settings[i][5:6])
        
        # the board writes back in bytes, not strings, hence the b'0' or b'1'.
        if settings[i][2:3] == b'0':
            counters[i].portA.active = []
        else:
            counters[i].portA.active = [0]
        
        if settings[i][3:4] == b'0':
            counters[i].portB.active = []
        else:
            counters[i].portB.active = [0]

        if settings[i][4:5] == b'0':
            counters[i].portC.active = []
        else:
            counters[i].portC.active = [0]

        if settings[i][5:6] == b'0':
            counters[i].portD.active = []
        else:
            counters[i].portD.active = [0]

# run the set_checkboxes function to give the checkboxes proper 
# intial values that reflect the current board settings
set_checkboxes()



# Create info labels that can warn the user if an error is made (you cannot clear a counter)
infoLabels = [None] * 8

# append blank text ' ' that can be changed later
for i in range(8):
    infoLabels[i] = PreText(text='', width = 400, height = widgetHeight)

    
# Create one horizontal box for each counter, which includes the port checkboxes
counterBoxes = [None] * 8

for i in range(8):
    counterBoxes[i] = row([counter_labels[i], 
                               counters[i].portA, 
                               counters[i].portB, 
                               counters[i].portC, 
                               counters[i].portD, 
                               blank2,
                               infoLabels[i]])
    


# create a horizontal box with the port column labels
portsBox = row([blank, portA, portB, portC, portD])

# display our row with button and info_sent label
buttonRow = row([blank3, set_button])


# create a column of text widgets to display the board settings
boardSettingsColumn = column([boardSettings[0],
                              boardSettings[1],
                              boardSettings[2],
                              boardSettings[3],
                              boardSettings[4],
                              boardSettings[5],
                              boardSettings[6],
                              boardSettings[7],
                              boardSettings[8],
                              boardSettings[9],
                              boardSettings[10],
                              boardSettings[11],
                              boardSettings[12]])

# create the layout of our interface including all the rows and columns above
counterInterface = row([column([portsBox, 
                          counterBoxes[0], 
                          counterBoxes[1], 
                          counterBoxes[2], 
                          counterBoxes[3],
                          counterBoxes[4],
                          counterBoxes[5],
                          counterBoxes[6],
                          counterBoxes[7],
                          buttonRow]), boardSettingsColumn])


# define a function that will be run every time the Set Counters button is clicked
def send_info():
    
    # loop through 8 times, one for each counter
    for i in range(8):
        # set the values A-D to 0
        A = 0
        B = 0
        C = 0
        D = 0
        # clear the warning label
        infoLabels[i].text = ''
        # check the checkbox values of portA - portD, and change respective values as needed
        if counters[i].portA.active == [0]:
            A = 1
        if counters[i].portB.active == [0]:
            B = 1
        if counters[i].portC.active == [0]:
            C = 1
        if counters[i].portD.active == [0]:
            D = 1
            
        # if the sum of A - D is at least 1 (aka at least one port box is checked/True)
        # then write the counter setting command to the board
        if A + B + C + D > 0:
            countingCommand = 'S'+ str(i) + str(int(A)) + str(int(B)) + str(int(C)) + str(int(D)) + '\n'
            s.write(countingCommand.encode())
        # else (if the sum of A - D is 0), warn the user that this is not permitted. 
        # Do not send command to board
        else:
            infoLabels[i].text = 'Counter ' + str(i) + ' must watch at least 1 port.'
    
    # run the set_checkboxes function to ensure that the values of the checkboxes 
    # reflect the current board settings
    set_checkboxes()


    
# call the send_info function when the button is clicked.
set_button.on_click(send_info)





# This portion of the code is for the Three Graphs


useSerial = True

# Set up the CD48 board
#if useSerial:
#    s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000) # May need to change the port number
    #s.write("S01100\n".encode()) 

    
    
# Initialize the data lists, time is the same for all three plots
# use a Column Data Source, intially all lists are empty

# Convention:
# single counts for G (gate) will be stored on Counter 0
# single counts for T (transmitted) will be stored on Counter 1
# single counts for R (reflected) will be stored on Counter 2
# coincidence counts for GT will be stored on Counter 3
# coincidence counts for GR will be stored on Counter 4
# three-fold coincidence counts for GTR will be stored on Counter 5

source = ColumnDataSource(dict(
    timeList=[],
    singleCountsG=[], 
    singleCountsT=[], 
    singleCountsR=[],
    coincidenceCountsGT=[],
    coincidenceCountsGR=[],
    coincidenceCountsGTR=[],
    accidentalCoincidencesGT=[],
    accidentalCoincidencesGR=[],
    accidentalCoincidencesGTR=[],
    ratioCountsGT=[],
    ratioCountsGR=[],
    ratioCountsGTR=[],
    g2=[],
    accidentalg2=[],
    adjustedg2=[]
    ))

sourceG2 = ColumnDataSource(dict(
    timeListG2=[],
    G2=[] 
    ))

sourceRatio = ColumnDataSource(dict(
    timeListRatio=[],
    Ratio=[] 
    ))


# just commenting these out for now, can add them back later

# Add Hover Tools
GHover = [
    ("Time", "@timeList"),
    ("Counts", "@singleCountsG")
]

THover = [
    ("Time", "@timeList"),
    ("Counts", "@singleCountsT")
]

RHover = [
    ("Time", "@timeList"),
    ("Counts", "@singleCountsR")
]

GTHover = [
    ("Time", "@timeList"),
    ("Total Counts", "@coincidenceCountsGT"),
    ("Accidental Counts", "@accidentalCoincidencesGT")
]

GRHover = [
    ("Time", "@timeList"),
    ("Total Counts", "@coincidenceCountsGR"),
    ("Accidental Counts", "@accidentalCoincidencesGR")
]

GTRHover = [
    ("Time", "@timeList"),
    ("Total Counts", "@coincidenceCountsGTR"),
    ("Accidental Counts", "@accidentalCoincidencesGTR")
]



# Create our 6 plots (3 singles, 3 coincidences)
singleCountsGPlot = figure(plot_height=360, plot_width=600, title="Single Counts Gate",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = GHover
                          )

singleCountsTPlot = figure(plot_height=380, plot_width=600, title="Single Counts Transmitted",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = THover
                          )

singleCountsRPlot = figure(plot_height=380, plot_width=600, title="Single Counts Reflected",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = RHover
                          )

coincidenceCountsGTPlot = figure(plot_height=360, plot_width=600, 
                                 title="Coincidence Counts Gate and Transmitted",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = GTHover
                                )

coincidenceCountsGRPlot = figure(plot_height=380, plot_width=600, 
                                 title="Coincidence Counts Gate and Reflected",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = GRHover
                                )

coincidenceCountsGTRPlot = figure(plot_height=380, plot_width=600, 
                                 title="Three-fold Coincidence Counts",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = GTRHover
                                 )

# Hover tools for plots with multiple lines are added later 
ratioPlot = figure(plot_height=360, plot_width=1000, 
                                 title="Coincidences to Single Counts Ratio",
              tools="crosshair,pan,reset,save,wheel_zoom"
                                 ) 

g2Plot = figure(plot_height=380, plot_width=1000, 
                                 title="g2",
              tools="crosshair,pan,reset,save,wheel_zoom"
                                 )






# Set up slider for user to choose counting time (100ms to 5s)
countingTimeSlider = Slider(start=0.1, end=5, value=1, step=0.1, title="Counting Time (s)", width = 200)
countingTime = countingTimeSlider.value # initialize the variable to store the counting interval


# Set up slider for user to choose how many points are displayed (10 to 100)
numberPointsSlider = Slider(start=10, end=100, value=20, step=5, title="Number of Points Displayed", 
                            width = 200)
numberPointsDisplayed = numberPointsSlider.value # initialize the variable to store the counting interval



# Make the plots display only the most recent "numberPointsDisplayed" points
# This way, the graph will scroll to show the newest points
# range_padding keeps the end points a certain distance from the edge of the graph
singleCountsGPlot.x_range.follow = "end"
singleCountsGPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
singleCountsGPlot.x_range.range_padding = 0.1
singleCountsGPlot.x_range.range_padding_units = 'absolute'
    
singleCountsTPlot.x_range.follow = "end"
singleCountsTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
singleCountsTPlot.x_range.range_padding = 0.1
singleCountsTPlot.x_range.range_padding_units = 'absolute'

singleCountsRPlot.x_range.follow = "end"
singleCountsRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
singleCountsRPlot.x_range.range_padding = 0.1
singleCountsRPlot.x_range.range_padding_units = 'absolute'
    
coincidenceCountsGTPlot.x_range.follow = "end"
coincidenceCountsGTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
coincidenceCountsGTPlot.x_range.range_padding = 0.1
coincidenceCountsGTPlot.x_range.range_padding_units = 'absolute'

coincidenceCountsGRPlot.x_range.follow = "end"
coincidenceCountsGRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
coincidenceCountsGRPlot.x_range.range_padding = 0.1
coincidenceCountsGRPlot.x_range.range_padding_units = 'absolute'

coincidenceCountsGTRPlot.x_range.follow = "end"
coincidenceCountsGTRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
coincidenceCountsGTRPlot.x_range.range_padding = 0.1
coincidenceCountsGTRPlot.x_range.range_padding_units = 'absolute'

ratioPlot.x_range.follow = "end"
ratioPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
ratioPlot.x_range.range_padding = 0.1
ratioPlot.x_range.range_padding_units = 'absolute'

g2Plot.x_range.follow = "end"
g2Plot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
g2Plot.x_range.range_padding = 0.1
g2Plot.x_range.range_padding_units = 'absolute'
        

# Connect the points of each plot with a line
singleCountsGPlot.line(x = 'timeList', y = 'singleCountsG', source = source, line_width=3, line_alpha=0.6,
                        line_color = 'red')
singleCountsTPlot.line(x = 'timeList', y = 'singleCountsT', source = source, line_width=3, line_alpha=0.6,
                        line_color = 'green')
singleCountsRPlot.line(x = 'timeList', y = 'singleCountsR', source = source, line_width=3, line_alpha=0.6,
                        line_color = 'blue')

coincidenceCountsGTPlot.line(x = 'timeList', y = 'coincidenceCountsGT', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'red')
coincidenceCountsGRPlot.line(x = 'timeList', y = 'coincidenceCountsGR', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'green')
coincidenceCountsGTRPlot.line(x = 'timeList', y = 'coincidenceCountsGTR', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'blue')

ratioPlot.line(x = 'timeList', y = 'ratioCountsGT', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'red', legend_label="GT")
ratioPlot.line(x = 'timeList', y = 'ratioCountsGR', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'green', legend_label="GR")
ratioPlot.line(x = 'timeList', y = 'ratioCountsGTR', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'blue', legend_label="GTR")



g2Plot.line(x = 'timeList', y = 'g2', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'purple', legend_label="G2")

g2Plot.line(x = 'timeList', y = 'adjustedg2', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'orchid',
                               legend_label="G2 (Adjusted)")


# Make each point of each plot a circle
singleCountsGPlot.circle(x = 'timeList', y = 'singleCountsG', source = source, size = 12, color = 'red')
singleCountsTPlot.circle(x = 'timeList', y = 'singleCountsT', source = source, size = 12, color = 'green')
singleCountsRPlot.circle(x = 'timeList', y = 'singleCountsR', source = source, size = 12, color = 'blue')

coincidenceCountsGTPlot.circle(x = 'timeList', y = 'coincidenceCountsGT', source = source, 
                               size = 12, color = 'red')
coincidenceCountsGRPlot.circle(x = 'timeList', y = 'coincidenceCountsGR', source = source, 
                               size = 12, color = 'green')
coincidenceCountsGTRPlot.circle(x = 'timeList', y = 'coincidenceCountsGTR', source = source, 
                               size = 12, color = 'blue')

# Create scatter plots for the ratio plot and add their associated hover tools
GTPoints = ratioPlot.circle(x = 'timeList', y = 'ratioCountsGT', source = source, 
                               size = 12, color = 'red', legend_label="GT")
ratioPlot.add_tools(HoverTool(renderers=[GTPoints], tooltips=[("Time","@timeList"), 
                                ("Ratio","@ratioCountsGT")], mode='mouse'))

GRPoints = ratioPlot.circle(x = 'timeList', y = 'ratioCountsGR', source = source, 
                               size = 12, color = 'green', legend_label="GR")
ratioPlot.add_tools(HoverTool(renderers=[GRPoints], tooltips=[("Time","@timeList"), 
                                ("Ratio","@ratioCountsGR")], mode='mouse'))

GTRPoints = ratioPlot.circle(x = 'timeList', y = 'ratioCountsGTR', source = source, 
                               size = 12, color = 'blue', legend_label="GTR")
ratioPlot.add_tools(HoverTool(renderers=[GTRPoints], tooltips=[("Time","@timeList"), 
                                ("Ratio","@ratioCountsGTR")], mode='mouse'))


g2Points = g2Plot.circle(x = 'timeList', y = 'g2', source = source, 
                               size = 12, color = 'purple', legend_label="G2")
g2Plot.add_tools(HoverTool(renderers=[g2Points], tooltips=[("Time","@timeList"),
                        ("g2","@g2"), ("g2 (Accidental)","@accidentalg2")], mode='mouse'))

adjustedg2Points = g2Plot.circle(x = 'timeList', y = 'adjustedg2', source = source, 
                               size = 12, color = 'orchid', 
                               legend_label="G2 (Adjusted)")
g2Plot.add_tools(HoverTool(renderers=[adjustedg2Points], tooltips=[("Time","@timeList"), 
                        ("g2 (Adjusted)","@adjustedg2")], mode='mouse'))



falseCircleG2 = g2Plot.circle(x = 'timeListG2', y = 'G2', source = sourceG2,
                            size=12, color = 'darkgrey')
    
g2Plot.add_tools(HoverTool(renderers=[falseCircleG2], tooltips=[("Time", "@timeListG2"), 
                        ("G2 Error","False Zero"), ("Adjusted G2 Error","False Zero")], mode='mouse'))


falseCircleRatio = ratioPlot.circle(x = 'timeListRatio', y = 'Ratio', source = sourceRatio,
                            size=12, color = 'darkgrey')
    
ratioPlot.add_tools(HoverTool(renderers=[falseCircleRatio], tooltips=[("Time", "@timeListRatio"), 
                        ("Error","False Zeros")], mode='mouse'))


# Specify the location of the legend
ratioPlot.legend.location = "top_left"
# Specify the location of the legend
g2Plot.legend.location = "top_left"

# Set y-axis labels for graphs
singleCountsGPlot.yaxis.axis_label = "Counts"
singleCountsTPlot.yaxis.axis_label = "Counts"
singleCountsRPlot.yaxis.axis_label = "Counts"

coincidenceCountsGTPlot.yaxis.axis_label = "Counts"
coincidenceCountsGRPlot.yaxis.axis_label = "Counts"
coincidenceCountsGTRPlot.yaxis.axis_label = "Counts"

ratioPlot.yaxis.axis_label = "Ratio"

g2Plot.yaxis.axis_label = "g2"


# Set x-axis labels for bottom-most graphs

#singleCountsGPlot.xaxis.axis_label = "Time (s)"
singleCountsTPlot.xaxis.axis_label = "Time (s)"
singleCountsRPlot.xaxis.axis_label = "Time (s)"

#coincidenceCountsGTPlot.xaxis.axis_label = "Time (s)"
coincidenceCountsGRPlot.xaxis.axis_label = "Time (s)"
coincidenceCountsGTRPlot.xaxis.axis_label = "Time (s)"


#ratioPlot.xaxis.axis_label = "Time (s)"

g2Plot.xaxis.axis_label = "Time (s)"


# set y axis tick label size
singleCountsGPlot.yaxis.major_label_text_font_size = "18pt"
singleCountsTPlot.yaxis.major_label_text_font_size = "18pt"
singleCountsRPlot.yaxis.major_label_text_font_size = "18pt"
coincidenceCountsGTPlot.yaxis.major_label_text_font_size = "18pt"
coincidenceCountsGRPlot.yaxis.major_label_text_font_size = "18pt"
coincidenceCountsGTRPlot.yaxis.major_label_text_font_size = "18pt"
ratioPlot.yaxis.major_label_text_font_size = "18pt"
g2Plot.yaxis.major_label_text_font_size = "18pt"



# Set up START/STOP button, to start and stop counting
startStopButtonGroup = RadioButtonGroup(labels=["START", "STOP"], active=1, width = 200)

# Set up the reset button, to clear the graph of all data
resetGraphButton = Button(label="Reset", width = 200)

# Set up the save datda button, to export the data to a csv file
saveDataButton = Button(label="Save Data", width = 200)

# Set up the autoscaling toggle button
autoscalingText = PreText(text = 'Autoscaling:', width=200)
autoscalingToggle = RadioButtonGroup(labels=["ON","OFF"], active = 0, width = 200)
autoscalingActive = autoscalingToggle.active # keep track of the mode

zoomSlider = Slider(start=1, end=100, value=33, step=1, title="Zoom Amount (%)", width = 200)

zoomMaxText = PreText(text = 'Max', width=40)
zoomMinText = PreText(text = 'Min', width=160)

# Make a label reminding counter settings
counterSettings = PreText(text = ''' 
Gate --> Port A
Transmitted --> Port B
Reflected --> Port C

Counter Settings should be
Counter 0: A
Counter 1: B
Counter 2: C
Counter 3: AB
Counter 4: AC
Counter 5: ABC''', width = 200)



    

# keep a running count of time elapsed, start at 0. This will help inform x-axis values.
runningTime = 0

# keep a count, add one each time update_data is called. 
# use this to reset y-axis scaling periodically
rescaleCountG = 0
rescaleCountT = 0
rescaleCountR = 0
rescaleCountGT = 0
rescaleCountGR = 0
rescaleCountGTR = 0
rescaleCountRatio = 0
rescaleCountG2 = 0

# use these to keep track of the end of y_range for all plots
yStartG = -100
yEndG = 1000

yStartT = -100
yEndT = 1000

yStartR = -100
yEndR = 1000

yStartGT = -100
yEndGT = 1000

yStartGR = -100
yEndGR = 1000

yStartGTR = -100
yEndGTR = 1000

yStartRatio = -0.1
yEndRatio = 1.2

yStartG2 = -1
yEndG2 = 3



# Set y_ranges for all plots
# if you do not do this, you will not be able to alter it later

singleCountsGPlot.y_range = Range1d(yStartG, yEndG)

singleCountsTPlot.y_range = Range1d(yStartT, yEndT)

singleCountsRPlot.y_range = Range1d(yStartR, yEndR)

coincidenceCountsGTPlot.y_range = Range1d(yStartGT, yEndGT)

coincidenceCountsGRPlot.y_range = Range1d(yStartGR, yEndGR)

coincidenceCountsGTRPlot.y_range = Range1d(yStartGTR, yEndGTR)

ratioPlot.y_range = Range1d(yStartRatio, yEndRatio)

g2Plot.y_range = Range1d(yStartG2, yEndG2)


# define global lists for last 5 data points
last5G = np.zeros(5)
last5T = np.zeros(5)
last5R = np.zeros(5)
last5GT = np.zeros(5)
last5GR = np.zeros(5)
last5GTR = np.zeros(5)
last5Ratio = np.zeros(15)
last5G2 = np.zeros(10)

# define false zeros booleans
falseZeroG2 = False
falseZeroRatios = False

# Define a fuction to call when the current document (curdoc) has periodic callback on
# This function appends new data to our plots through the stream data command
def update_data():
    
    # retrieve the value of desired counting time
    countingTime = countingTimeSlider.value
    
    # use the global variable
    global rescaleCountG
    global rescaleCountT
    global rescaleCountR
    global rescaleCountGT
    global rescaleCountGR
    global rescaleCountGTR
    global rescaleCountRatio
    global rescaleCountG2
    
    global runningTime
    
    global autoscalingActive
    
    global falseZeroG2
    global falseZeroRatios
    
    # add the counting time to our running time
    runningTime = runningTime + countingTime
    
    # retrieve data from the coincidence counting board and store them in a list 
    s.write("c\n".encode())
    serialData = s.readline()
    counterData = [int(x) for x in serialData.decode('ascii').rstrip().split(' ')]
    
    # rename the obtained data
    currentG = counterData[0]
    currentT = counterData[1]
    currentR = counterData[2]
    currentGT = counterData[3]
    currentGR = counterData[4]
    currentGTR = counterData[5]
    
    # accidental = N1*N2*deltaT / dwell time
    currentAccidentalGT = (currentG*currentT*25*(10**(-9)))/(countingTime)
    currentAccidentalGR = (currentG*currentR*25*(10**(-9)))/(countingTime)
    
    numerator = (currentGT*currentR + currentGR*currentT)*25*(10**(-9))
    currentAccidentalGTR = (numerator)/(countingTime)
    
    # adjusted = total - accidental
    #adjustedCoincidenceCounts = counterData[2] - calculatedAccidental
    
    # ratio = Nc / NG
    # avoid dividing by zero
    if currentG > 0:
        currentRatioGT = currentGT/currentG
        currentRatioGR = currentGR/currentG
        currentRatioGTR = currentGTR/currentG
    else:
        #nan = float('nan')
        currentRatioGT = 0
        currentRatioGR = 0
        currentRatioGTR = 0
        # need to alert the user that this value is not a number
        falseZeroRatios = True
        
    # g2 = NGTR * NG / (NGT * NGR)
    # avoid dividing by zero 
    if currentGT*currentGR > 0:
        
        currentG2 = (currentGTR * currentG) / (currentGT * currentGR)
        
        a = (currentG*currentR*25*(10**(-9)))/(currentGR*countingTime)
        b = (currentG*currentT*25*(10**(-9)))/(currentGT*countingTime)
        
        currentAccidentalG2 = a + b
        
    else:
        #nan = float('nan')
        currentG2 = 0 
        currentAccidentalG2 = 0
        
        # need to alert the user that this value is not a number
        falseZeroG2 = True
        
    currentAdjustedG2 = currentG2 - currentAccidentalG2
    
    # update the data source
    new_data = dict(
        timeList=[runningTime],
        singleCountsG=[currentG],
        singleCountsT=[currentT],
        singleCountsR=[currentR],
        coincidenceCountsGT=[currentGT],
        coincidenceCountsGR=[currentGR],
        coincidenceCountsGTR=[currentGTR],
        accidentalCoincidencesGT = [currentAccidentalGT],
        accidentalCoincidencesGR = [currentAccidentalGR],
        accidentalCoincidencesGTR = [currentAccidentalGTR],
        ratioCountsGT=[currentRatioGT],
        ratioCountsGR=[currentRatioGR],
        ratioCountsGTR=[currentRatioGTR],
        g2=[currentG2],
        accidentalg2=[currentAccidentalG2],
        adjustedg2 = [currentAdjustedG2]
        )
    
    # Stream the new data to the plots.
    source.stream(new_data)
    
    if falseZeroG2:
        
        new_data_G2 = dict(
            timeListG2=[runningTime],
            G2=[currentG2]
            )
        
        sourceG2.stream(new_data_G2)
        
        falseZeroG2 = False
        
    if falseZeroRatios:
        
        new_data_Ratio = dict(
            timeListRatio=[runningTime],
            Ratio=[currentRatioGTR]
            )
        
        sourceRatio.stream(new_data_Ratio)
        
        falseZeroRatios = False
    
    
    # use global variable
    global yStartG
    global yStartT
    global yStartR
    global yStartGT
    global yStartGR
    global yStartGTR
    global yStartRatio
    global yStartG2
    
    global yEndG
    global yEndT
    global yEndR
    global yEndGT
    global yEndGR
    global yEndGTR
    global yEndRatio
    global yEndG2
    
    # use global last five points lists
    global last5G
    global last5T
    global last5R
    global last5GT
    global last5GR
    global last5GTR
    global last5Ratio
    global last5G2
    
    
    # update the list of the last 5 data points
    
    # shift every element of the list back one index
    for i in np.arange(4):
        last5G[i] = last5G[i+1]       
    # make the most recent point the 5th element of the list
    last5G[4] = currentG
    
    for i in np.arange(4):
        last5T[i] = last5T[i+1]
    last5T[4] = currentT

    for i in np.arange(4):
        last5R[i] = last5R[i+1]
    last5R[4] = currentR
   
    for i in np.arange(4):
        last5GT[i] = last5GT[i+1]
    last5GT[4] = currentGT
        
    for i in np.arange(4):
        last5GR[i] = last5GR[i+1]
    last5GR[4] = currentGR
  
    for i in np.arange(4):
        last5GTR[i] = last5GTR[i+1]
    last5GTR[4] = currentGTR
   
    for i in np.arange(4):
        last5G2[i] = last5G2[i+1]
    for i in np.arange(4):
        last5G2[i+5] = last5G2[i+6]
    
    last5G2[4] = currentG2
    last5G2[9] = currentAdjustedG2

    # the last5Ratio list stores the most recent ratio data. 0-4: GT ratio. 5-9: GR ratio. 10-14: GTR ratio.
    for i in np.arange(4):
        last5Ratio[i] = last5Ratio[i+1]
    for i in np.arange(4):
        last5Ratio[i+5] = last5Ratio[i+6]
    for i in np.arange(4):
        last5Ratio[i+10] = last5Ratio[i+11]
        
    last5Ratio[4] = currentRatioGT
    last5Ratio[9] = currentRatioGR
    last5Ratio[14] = currentRatioGTR

    
    # Rescale y-axes only when the autoscaling is on
    if autoscalingActive == 0:
        
        # map 1-100 on to 0.3-0.003
        zoomAmount = (101 - zoomSlider.value) / 1000 * 3
        
        # check for spikes outside of y_range on all four plots
        # if new point y value is outside of y_range, then change range

        if currentG > yEndG or currentG < yStartG:   
            # set the rescale count to 5 so that the plot will be rescaled immediately to show all 
            # last 5 points
            rescaleCountG = 5

        if currentT > yEndT or currentT < yStartT:   
            rescaleCountT = 5

        if currentR > yEndR or currentR < yStartR:   
            rescaleCountR = 5

        if currentGT > yEndGT or currentGT < yStartGT:   
            rescaleCountGT = 5

        if currentGR > yEndGR or currentGR < yStartGR:   
            rescaleCountGR = 5

        if currentGTR > yEndGTR or currentGTR < yStartGTR:   
            rescaleCountGTR = 5
            
        maxRatio = max([currentRatioGT, currentRatioGR, currentRatioGTR])
        minRatio = min([currentRatioGT, currentRatioGR, currentRatioGTR])

        # if any ratio goes above y range, use max & min value to extend y range
        if maxRatio > yEndRatio or minRatio < yStartRatio:  
            rescaleCountRatio = 5
        
        # G2 will always be >= to adjusted g2, so no need to include a max function here
        if currentG2 > yEndG2 or currentG2 < yStartG2:   
            rescaleCountG2 = 5


        # find the max values of the last 5 data points
        recentMaxG = max(last5G)
        recentMaxT = max(last5T)
        recentMaxR = max(last5R)
        recentMaxGT = max(last5GT)
        recentMaxGR = max(last5GR)
        recentMaxGTR = max(last5GTR)
        recentMaxRatio = max(last5Ratio)
        recentMaxG2 = max(last5G2)
        
        recentMinG = min(last5G)
        recentMinT = min(last5T)
        recentMinR = min(last5R)
        recentMinGT = min(last5GT)
        recentMinGR = min(last5GR)
        recentMinGTR = min(last5GTR)
        recentMinRatio = min(last5Ratio)
        recentMinG2 = min(last5G2) 


        # if reset y axis count is greater than or equal to 5, we rescale the 
        # y axis based on the most recent 5 points

        if rescaleCountG >= 5:
            rescaleCountG = 0
            #sourceDataText.text = ' '.join(map(str, max(source.data['singleCountsA'][-5:])))
            # if the recent max value is greater than 0:
            if recentMaxG > 0:
                # reset the y-axis scaling to show recent max plus 0.2 * recent max
                yEndG = recentMaxG*(1+zoomAmount)
            else:
                # reset the upper limit to 1000
                yEndG = 1000
                
            if recentMinG > 0:
                # reset the y-axis scaling to show recent min minus 0.2 * recent min
                yStartG = recentMinG*(1-zoomAmount)
            else:
                # reset the lower limit to -100
                yStartG = -100

        if rescaleCountT >= 5: 
            rescaleCountT = 0
            if recentMaxT > 0:
                yEndT = recentMaxT*(1+zoomAmount)
            else:
                yEndT = 1000
                
            if recentMinT > 0:
                yStartT = recentMinT*(1-zoomAmount)
            else:
                yStartT = -100

        if rescaleCountR >= 5: 
            rescaleCountR = 0
            if recentMaxR > 0:
                yEndR = recentMaxR*(1+zoomAmount)
            else:
                yEndR = 1000
            
            if recentMinR > 0:
                yStartR = recentMinR*(1-zoomAmount)
            else:
                yStartR = -100

        if rescaleCountGT >= 5:
            rescaleCountGT = 0
            if recentMaxGT > 0:
                yEndGT = recentMaxGT*(1+zoomAmount)
            else:
                yEndGT = 10
                
            if recentMinGT > 0:
                yStartGT = recentMinGT*(1-zoomAmount)
            else:
                yStartGT = -10

        if rescaleCountGR >= 5: 
            rescaleCountGR = 0
            if recentMaxGR > 0:
                yEndGR = recentMaxGR*(1+zoomAmount)
            else:
                yEndGR = 10
                
            if recentMinGR > 0:
                yStartGR = recentMinGR*(1-zoomAmount)
            else:
                yStartGR = -10

        if rescaleCountGTR >= 5:
            rescaleCountGTR = 0
            if recentMaxGTR > 0:
                yEndGTR = recentMaxGTR*(1+zoomAmount)
            else:
                yEndGTR = 10
                
            if recentMinGTR > 0:
                yStartGTR = recentMinGTR*(1-zoomAmount)
            else:
                yStartGTR = -10

        if rescaleCountRatio >= 5:
            rescaleCountRatio = 0
            if recentMaxRatio > 0:
                yEndRatio = recentMaxRatio*1.2
            else:
                yEndRatio = 0.01
                
            if recentMinRatio > 0:
                yStartRatio = recentMinRatio*0.8
            else:
                yStartRatio = -0.01

        if rescaleCountG2 >= 5:
            rescaleCountG2 = 0
            if recentMaxG2 > 0:
                yEndG2 = recentMaxG2*1.2
            else:
                yEndG2 = 2
                
            if recentMinG2 > 0:
                yStartG2 = recentMinG2*0.8
            else:
                yStartG2 = -0.1


        # update the y axis range
        singleCountsGPlot.y_range.update(start = yStartG, end = yEndG)
        singleCountsTPlot.y_range.update(start = yStartT, end = yEndT)
        singleCountsRPlot.y_range.update(start = yStartR, end = yEndR)
        coincidenceCountsGTPlot.y_range.update(start = yStartGT, end = yEndGT)
        coincidenceCountsGRPlot.y_range.update(start = yStartGR, end = yEndGR)
        coincidenceCountsGTRPlot.y_range.update(start = yStartGTR, end = yEndGTR)
        ratioPlot.y_range.update(start = yStartRatio, end = yEndRatio)
        g2Plot.y_range.update(start = yStartG2, end = yEndG2)



        # add 1 to reset y-axis counter
        rescaleCountG = rescaleCountG + 1
        rescaleCountT = rescaleCountT + 1
        rescaleCountR = rescaleCountR + 1
        rescaleCountGT = rescaleCountGT + 1
        rescaleCountGR = rescaleCountGR + 1
        rescaleCountGTR = rescaleCountGTR + 1
        rescaleCountRatio = rescaleCountRatio + 1
        rescaleCountG2 = rescaleCountG2 + 1




# Define a variable callback_id, which is attached to the periodic_callback function
callback_id = None


# Define a function to be called when the START/STOP button changes value
def change_periodic_callback(new):
    # use global variable
    global callback_id
    
    # if the START button is on
    if startStopButtonGroup.active == 0:
        
        # clear the counters
        s.write("c\n".encode())
        serialData = s.readline()
        
        # Add a periodic_callback, the callback interval is stored in the variable countingTime
        #global countingTime
        countingTime = countingTimeSlider.value
        callback_id = curdoc().add_periodic_callback(update_data, countingTime*1000)
    
    # else if the STOP button is on
    elif startStopButtonGroup.active == 1:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
        

# Define a function to be called when the reset button is pressed.   
def reset_plot():
    
    # set the running time back to 0
    global runningTime
    runningTime = 0
    
    global resetYAxisCount
    resetYAxisCount = 0
    
    global last5G
    global last5T
    global last5R
    global last5GT
    global last5GR
    global last5GTR
    global last5Ratio
    global last5G2
    
    last5G = np.zeros(5)
    last5T = np.zeros(5)
    last5R = np.zeros(5)
    last5GT = np.zeros(5)
    last5GR = np.zeros(5)
    last5GTR = np.zeros(5)
    last5Ratio = np.zeros(15)    
    last5G2 = np.zeros(10)
    
    # empty all data lists in source
    source.data = {k: [] for k in source.data}
    
    sourceG2.data = {k: [] for k in sourceG2.data}
    
    sourceRatio.data = {k: [] for k in sourceRatio.data}
    

def change_callback_period(attr, old, new):
    # use global variable
    global callback_id
    
    global numberPointsDisplayed
    
    global countingTime
    
    countingTime = countingTimeSlider.value
    
    # if the START button is on
    if startStopButtonGroup.active == 0:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
        
        # Add a periodic_callback, the callback interval is stored in the variable countingTime
        callback_id = curdoc().add_periodic_callback(update_data, countingTime*1000)
        
    # How many points should be displayed on screen at a time. Can turn this into a user input widget
    numberPointsDisplayed = numberPointsSlider.value


    # Make the plots display only the most recent "numberPointsDisplayed" points
    # This way, the graph will scroll to show the newest points
    singleCountsGPlot.x_range.follow = "end"
    singleCountsGPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    singleCountsTPlot.x_range.follow = "end"
    singleCountsTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    singleCountsRPlot.x_range.follow = "end"
    singleCountsRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGTPlot.x_range.follow = "end"
    coincidenceCountsGTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGRPlot.x_range.follow = "end"
    coincidenceCountsGRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGTRPlot.x_range.follow = "end"
    coincidenceCountsGTRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
    
    g2Plot.x_range.follow = "end"
    g2Plot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    ratioPlot.x_range.follow = "end"
    ratioPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
 


        
        
def change_number_points(attr, old, new):
    
    global numberPointsDisplayed
    global countingTime
    
    # retrieve slider values
    numberPointsDisplayed = numberPointsSlider.value 
    countingTime = countingTimeSlider.value


    # Make the plots display only the most recent "numberPointsDisplayed" points
    # This way, the graph will scroll to show the newest points
    singleCountsGPlot.x_range.follow = "end"
    singleCountsGPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    singleCountsTPlot.x_range.follow = "end"
    singleCountsTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    singleCountsRPlot.x_range.follow = "end"
    singleCountsRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGTPlot.x_range.follow = "end"
    coincidenceCountsGTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGRPlot.x_range.follow = "end"
    coincidenceCountsGRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGTRPlot.x_range.follow = "end"
    coincidenceCountsGTRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
    
    g2Plot.x_range.follow = "end"
    g2Plot.x_range.follow_interval = (numberPointsDisplayed)*countingTime    

    ratioPlot.x_range.follow = "end"
    ratioPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
    
    
def change_autoscaling_mode(new):
    
    global autoscalingActive
    
    # update the value of autoscalingActive
    autoscalingActive = autoscalingToggle.active
    
    # update the label on the toggle button
    if autoscalingActive == 1:
        #autoscalingToggle.label = "Autoscaling ON"
        #else:
        #autoscalingToggle.label = "Autoscaling OFF"
        
        zoomAmount = (101 - zoomSlider.value) / 1000 * 3
        
        # use global variable
        global yStartG
        global yStartT
        global yStartR
        global yStartGT
        global yStartGR
        global yStartGTR

        global yEndG
        global yEndT
        global yEndR
        global yEndGT
        global yEndGR
        global yEndGTR
        
        # use global last five points lists
        global last5G
        global last5T
        global last5R
        global last5GT
        global last5GR
        global last5GTR
        
        # update the y range in terms of the average value of the most recet 5 points
        # plots with multiple lines are not changed
        # only update if counts are not all zeros, otherwise do nothing
        if np.mean(last5G) > 0:
            yStartG = (1-zoomAmount) * np.mean(last5G)
            yEndG = (1+zoomAmount) * np.mean(last5G)
        
        if np.mean(last5T) > 0:
            yStartT = (1-zoomAmount) * np.mean(last5T)
            yEndT = (1+zoomAmount) * np.mean(last5T)
        
        if np.mean(last5R) > 0:
            yStartR = (1-zoomAmount) * np.mean(last5R)
            yEndR = (1+zoomAmount) * np.mean(last5R)
            
        if np.mean(last5GT) > 0:
            yStartGT = (1-zoomAmount) * np.mean(last5GT)
            yEndGT = (1+zoomAmount) * np.mean(last5GT)
            
        if np.mean(last5GR) > 0:
            yStartGR = (1-zoomAmount) * np.mean(last5GR)
            yEndGR = (1+zoomAmount) * np.mean(last5GR)
            
        if np.mean(last5GTR) > 0:
            yStartGTR = (1-zoomAmount) * np.mean(last5GTR)
            yEndGTR = (1+zoomAmount) * np.mean(last5GTR)
            
            
        
        # update the y axis range
        singleCountsGPlot.y_range.update(start = yStartG, end = yEndG)
        singleCountsTPlot.y_range.update(start = yStartT, end = yEndT)
        singleCountsRPlot.y_range.update(start = yStartR, end = yEndR)
        coincidenceCountsGTPlot.y_range.update(start = yStartGT, end = yEndGT)
        coincidenceCountsGRPlot.y_range.update(start = yStartGR, end = yEndGR)
        coincidenceCountsGTRPlot.y_range.update(start = yStartGTR, end = yEndGTR)

        
        
def change_zoom(attr, old, new):
    
    if autoscalingActive == 1:
        
        zoomAmount = (101 - zoomSlider.value) / 1000 * 3
        
        # use global variable
        global yStartG
        global yStartT
        global yStartR
        global yStartGT
        global yStartGR
        global yStartGTR

        global yEndG
        global yEndT
        global yEndR
        global yEndGT
        global yEndGR
        global yEndGTR
        
        # use global last five points lists
        global last5G
        global last5T
        global last5R
        global last5GT
        global last5GR
        global last5GTR
        
        # update the y range in terms of the average value of the most recet 5 points
        # plots with multiple lines are not changed
        # only update if counts are not all zeros, otherwise do nothing
        if np.mean(last5G) > 0:
            yStartG = (1-zoomAmount) * np.mean(last5G)
            yEndG = (1+zoomAmount) * np.mean(last5G)
        
        if np.mean(last5T) > 0:
            yStartT = (1-zoomAmount) * np.mean(last5T)
            yEndT = (1+zoomAmount) * np.mean(last5T)
        
        if np.mean(last5R) > 0:
            yStartR = (1-zoomAmount) * np.mean(last5R)
            yEndR = (1+zoomAmount) * np.mean(last5R)
            
        if np.mean(last5GT) > 0:
            yStartGT = (1-zoomAmount) * np.mean(last5GT)
            yEndGT = (1+zoomAmount) * np.mean(last5GT)
            
        if np.mean(last5GR) > 0:
            yStartGR = (1-zoomAmount) * np.mean(last5GR)
            yEndGR = (1+zoomAmount) * np.mean(last5GR)
            
        if np.mean(last5GTR) > 0:
            yStartGTR = (1-zoomAmount) * np.mean(last5GTR)
            yEndGTR = (1+zoomAmount) * np.mean(last5GTR)
            
            
        
        # update the y axis range
        singleCountsGPlot.y_range.update(start = yStartG, end = yEndG)
        singleCountsTPlot.y_range.update(start = yStartT, end = yEndT)
        singleCountsRPlot.y_range.update(start = yStartR, end = yEndR)
        coincidenceCountsGTPlot.y_range.update(start = yStartGT, end = yEndGT)
        coincidenceCountsGRPlot.y_range.update(start = yStartGR, end = yEndGR)
        coincidenceCountsGTRPlot.y_range.update(start = yStartGTR, end = yEndGTR)
        

    
# When the START/STOP button is clicked, call the change_periodic_callback function
startStopButtonGroup.on_click(change_periodic_callback)


# When the reset button is clicked, call the reset_plot function
resetGraphButton.on_click(reset_plot)

countingTimeSlider.on_change('value', change_callback_period)

# When the save data button is clicked, export the data into a csv file
saveDataButton.js_on_click(CustomJS(args=dict(source=source),
                            code=open(join(dirname(__file__), "download.js")).read()))

# When number of points is changed, change follow interval
numberPointsSlider.on_change('value', change_number_points)

# When the autoscaling toggle button is clicked, call the change_autoscaling_mode function
autoscalingToggle.on_click(change_autoscaling_mode)

# When zoom value is changed, change zoom amount
zoomSlider.on_change('value', change_zoom)

# Set up layouts and add to document

zoomTextRow = row([zoomMinText, zoomMaxText])

# create a column of user input widgets
userInputWidgets = column(startStopButtonGroup, 
                          countingTimeSlider,
                          numberPointsSlider,
                          resetGraphButton,
                          saveDataButton,
                          autoscalingText,
                          autoscalingToggle,
                          zoomSlider,
                          zoomTextRow
                         )


# create a layout with the set counters interface from the first half of this code
setCountersLayout = layout([column([counterInterface, counterSettings])], sizing_mode='scale_both')

# create a column with our three plots
plotsColumn1 = column([coincidenceCountsGTPlot,
                       coincidenceCountsGRPlot
                      ])

plotsColumn11 = column([coincidenceCountsGTRPlot
                      ])

plotsColumn2 = column([singleCountsGPlot, 
                       singleCountsTPlot
                     ])

plotsColumn21 = column([singleCountsRPlot
                     ])

plotsColumn3 = column([ratioPlot, 
                       g2Plot
                     ])

plotsColumn4 = column([singleCountsGPlot, 
                       singleCountsTPlot
                      ])

plotsColumn41 = column([coincidenceCountsGTPlot
                      ])

plotsColumn5 = column([singleCountsGPlot, 
                       singleCountsRPlot
                      ])

plotsColumn51 = column([coincidenceCountsGRPlot
                      ])

# create a layout for the graphs interface, with the input widgets and the three plots
#displayCountsLayout = layout([[row([userInputWidgets, plotsColumn1, plotsColumn2, plotsColumn3])]] , 
#                             sizing_mode='scale_both')

# create a tab for the set counters interface
setCountersTab = Panel(child = setCountersLayout, 
                       title="Set Counters")



singlesCountsLayout = row([userInputWidgets, plotsColumn2, plotsColumn21], sizing_mode='scale_height')

coincidenceCountsLayout = row([userInputWidgets, plotsColumn1, plotsColumn11], sizing_mode='scale_height')

ratioG2Layout = row([userInputWidgets, plotsColumn3])

GTLayout = row([userInputWidgets, plotsColumn4, plotsColumn41], sizing_mode='scale_height')

GRLayout = row([userInputWidgets, plotsColumn5, plotsColumn51], sizing_mode='scale_height')

# create tabs for the graphs interface
singlesCountsTab = Panel(child = singlesCountsLayout, 
                         title="Single Counts")

coincidenceCountsTab = Panel(child = coincidenceCountsLayout, 
                         title="Coincidence Counts")

ratioG2Tab = Panel(child = ratioG2Layout, 
                         title="G2 and Ratios")

GTTab = Panel(child = GTLayout, 
                         title="Gate and transmitted")

GRTab = Panel(child = GRLayout, 
                         title="Gate and reflected")

# combine all tabs into one layout
currentDocumentTabs = Tabs(tabs=[ coincidenceCountsTab, singlesCountsTab, ratioG2Tab, setCountersTab, GTTab, GRTab])


# Add tabs to current document
curdoc().add_root(currentDocumentTabs)

curdoc().title = "Alignment Interface"